# CoreMS Tutorial
`CoreMS` framework for mass spectrum

### 1. Create the reader obj 

In [43]:
from pathlib import Path
from corems.mass_spectrum.input.boosterHDF import ReadHDF_BoosterMassSpectrum
from corems.mass_spectrum.input.massList import ReadMassList
from corems.transient.input.BrukerSolarix import ReadBrukerSolarix

file_location = Path.cwd() / "tests/tests_data/" / "ESI_NEG_ESFA.ascii"
polarity = -1
reader_obj = ReadMassList(file_location, delimiter="  ", isCentroid=True)
#reader_obj = ReadMassList(file_location, delimiter=",")
#reader_obj = ReadMassList(file_location, delimiter="\t")
#reader_obj = ReadHDF_Booster(file_location, polariy) #acess MassSpec Obj  
reader_obj.file_location

### 2. Create the mass spectrum obj

In [44]:
mass_spectrum = reader_obj.get_mass_spectrum(polarity, auto_process=True)
mass_spectrum.mz_exp, mass_spectrum.abundance


### 3. Access mass spectrum peak objs

In [45]:
kb = {'C': 1, 'H': 2}

for mspeak in mass_spectrum.sort_by_abundance():
    
    print(mspeak.mz_exp, mspeak.abundance, mspeak.resolving_power, 
          mspeak.signal_to_noise, mspeak.kmd, mspeak.is_assigned)
    
    mspeak.change_kendrick_base(kb)



### 4. Access molecular formula  objs

In [46]:
for mspeaks in mass_spectrum.sort_by_abundance():
   if mspeak.is_assigned: #or just if mspeak:
        for mf in mspeaks:
            print(mf.mz_theor,mf.to_string,mf.to_string_formatted)

### 5. Mass spectrum calibration

In [47]:
from corems.molecular_id.search.findOxigenPeaks import FindOxygenPeaks
from corems.encapsulation.settings.molecular_id.MolecularIDSettings import MolecularSearchSettings
from corems.mass_spectrum.calc.CalibrationCalc import MZDomain_Calibration

MolecularSearchSettings.error_method = 'None'
MolecularSearchSettings.min_mz_error = -10
MolecularSearchSettings.max_mz_error = 10
MolecularSearchSettings.mz_error_range = 1
MolecularSearchSettings.isProtonated = True
MolecularSearchSettings.isRadical= True
MolecularSearchSettings.isadduct = False 

MolecularSearchSettings.usedAtoms['C'] = (1,90)
MolecularSearchSettings.usedAtoms['H'] = (4,200)
MolecularSearchSettings.usedAtoms['O'] = (1,20)
MolecularSearchSettings.usedAtoms['N'] = (0,0)
MolecularSearchSettings.usedAtoms['S'] = (0,0)


find_formula_thread = FindOxygenPeaks(mass_spectrum)
find_formula_thread.run()

mspeaks_results = find_formula_thread.get_list_found_peaks()


#calibrate = MZDomain_Calibration(mass_spectrum, mspeaks_results)
#calibrate.ledford_inverted_calibration(iteration=False)
#calibrate.quadratic(iteration=False)

#remove results from calibration
#mass_spectrum.clear_molecular_formulas()


### 6. Molecular formula search

In [0]:
from corems.molecular_id.calc.ClusterFilter import ClusteringFilter
from corems.molecular_id.search.molecularFormulaSearch import SearchMolecularFormulas

ClusteringFilter().filter_kendrick(mass_spectrum)

MolecularSearchSettings.min_mz_error = -1
MolecularSearchSettings.max_mz_error = 1
MolecularSearchSettings.mz_error_range = 1
MolecularSearchSettings.mz_error_average = 0
MolecularSearchSettings.min_abun_error = -30 # percentage 
MolecularSearchSettings.max_abun_error = 70 # percentage 

MolecularSearchSettings.isProtonated = True 
MolecularSearchSettings.isRadical= True 
MolecularSearchSettings.isadduct = True 

MolecularSearchSettings.usedAtoms['C'] = (1,90)
MolecularSearchSettings.usedAtoms['H'] = (4,200)
MolecularSearchSettings.usedAtoms['O'] = (1,8)
MolecularSearchSettings.usedAtoms['N'] = (0,0)
MolecularSearchSettings.usedAtoms['S'] = (0,0)

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)



### 7. Export results

In [0]:
from corems.mass_spectrum.output.export import MassSpecExport

exportMS= MassSpecExport('neg_esi_srfa_1ppm_test', mass_spectrum)
exportMS.run()

exportMS.output_type = 'csv'
exportMS.run()

exportMS.output_type = 'pandas'
exportMS.run()

df = exportMS.get_pandas_df()
print( df.head())